In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()



,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


In [4]:
df['is_male'].value_counts()


True     5150
False    4850
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [8]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [9]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [10]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6670 samples, validate on 742 samples
Epoch 1/10
6670/6670 [==============================] - 2s 295us/sample - loss: 3.8414 - mae: 1.2192 - mse: 3.8414 - val_loss: 2.0961 - val_mae: 1.1757 - val_mse: 2.0961
Epoch 2/10
6670/6670 [==============================] - 1s 160us/sample - loss: 1.3527 - mae: 0.9045 - mse: 1.3527 - val_loss: 1.1350 - val_mae: 0.8330 - val_mse: 1.1350
Epoch 3/10
6670/6670 [==============================] - 1s 188us/sample - loss: 1.3088 - mae: 0.8954 - mse: 1.3088 - val_loss: 3.6280 - val_mae: 1.6542 - val_mse: 3.6280
Epoch 4/10
6670/6670 [==============================] - 1s 148us/sample - loss: 1.3195 - mae: 0.8980 - mse: 1.3195 - val_loss: 1.1556 - val_mae: 0.8401 - val_mse: 1.1556
Epoch 5/10
6670/6670 [==============================] - 1s 140us/sample - loss: 1.2658 - mae: 0.8804 - mse: 1.2658 - val_loss: 1.1153 - val_mae: 0.8268 - val_mse: 1.1153
Epoch 6/10
6670/6670 [==============================] - 1s 163us/sample - loss: 1.2498 - mae: 0.8736 - 

In [11]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [12]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  5.964186
Actual val:  5.62619692624

Predicted val:  7.1941967
Actual val:  7.7492485093

Predicted val:  6.518556
Actual val:  7.12534030784

Predicted val:  7.0144725
Actual val:  7.5618555866

Predicted val:  6.0621967
Actual val:  5.56226287026

Predicted val:  6.443763
Actual val:  5.62399230362

Predicted val:  5.234529
Actual val:  7.8374334140999995

Predicted val:  6.596723
Actual val:  7.25100379718

Predicted val:  6.8847256
Actual val:  9.0169065158

Predicted val:  5.2936482
Actual val:  4.12484892202



In [13]:
wit_data = pd.concat([x_test, y_test], axis=1)


In [14]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [15]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…